In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import glob
import time
from matplotlib.colors import LogNorm
import h5py
import matplotlib.pylab as pylab
import scipy.spatial.distance as scp
import pyemma 
import tqdm
from mpl_toolkits import mplot3d
import extq
import warnings
import matplotlib as mpl
from scipy import stats

import matplotlib.transforms as mtransforms
%matplotlib notebook 

In [ ]:
%matplotlib notebook

In [ ]:
def plot_pmf(x,y,weights,kT,xedges,yedges,
                title=None,xlabel='x',ylabel='y',label=None,ax=None,vmin=None,vmax=None,
                colorbar=True,cmap='cividis',contours=True,pcolor=True,clines=np.linspace(0,50,51),**kwargs):
    
    """Compute and plot a 2D PMF

    This function histograms the data, weighted by the change of measure, 
    to generate a PMF

    Parameters
    ----------
    x, y : ndarray or list/tuple of ndarray
        Coordinates of each frame.
    weights : ndarray or list/tuple of ndarray
        Weight or value of each frame (change of measure)
    kt: float
        Value for Boltzmann constant and temperature (choice of units)
    xedges, yedges : (nx+1,ny+1) ndarray
        The edges of the PMF 
    plotting parameters

    Returns
    -------
    hist : (nx, ny) ndarray
        Histogram of the 2D PMF
    """
    
    # bin values
    hist, _, _ = np.histogram2d(
        x.ravel(), y.ravel(), bins=(xedges, yedges), weights=weights.ravel()
    )
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        hist = -kT * np.log(hist)
    hist -= np.nanmin(hist)
    if vmin is not None or vmax is not None:
        if vmin is not None:
            hist[hist < vmin] = np.nan
        if vmax is not None:
            hist[hist > vmax] = np.nan
    # plot the PMF in bins
    if ax is None:
        ax = plt.gca()
    fig = ax.get_figure()
    if pcolor:
        pcm = ax.pcolormesh(xedges, yedges, hist.T, shading="flat", cmap=cmap, **kwargs)
        if colorbar:
            cbar = fig.colorbar(pcm, ax=ax)
        if label is not None:
            cbar.set_label(label, rotation=-90, va="baseline")
    ax.set_xlim(xedges[0], xedges[-1])
    ax.set_ylim(yedges[0], yedges[-1])
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    #Draw contours if desired
    if contours:
        centerx =(xedges[1:]+xedges[:-1])/2
        centery =(yedges[1:]+yedges[:-1])/2
        if pcolor:
            ax.contour(centerx, centery, hist.T, levels=clines, colors='black', linestyles='solid', linewidths=1)
        else: 
            ax.contour(centerx, centery, hist.T, levels=clines, colors='black', linestyles='solid', linewidths=0.5)
    return hist

In [ ]:
def make_pmf(x,y,weights,kT,xedges,yedges):
    
    """Compute  2D PMF (no plot)

    This function histograms the data, weighted by the change of measure, 
    to generate a PMF

    Parameters
    ----------
    x, y : ndarray or list/tuple of ndarray
        Coordinates of each frame.
    weights : ndarray or list/tuple of ndarray
        Weight or value of each frame (change of measure)
    kt: float
        Value for Boltzmann constant and temperature (choice of units)
    xedges, yedges : (nx+1,ny+1) ndarray
        The edges of the PMF 

    Returns
    -------
    hist : (nx, ny) ndarray
        Histogram of the 2D PMF
    """
    
    # bin values
    hist, _, _ = np.histogram2d(
        x.ravel(), y.ravel(), bins=(xedges, yedges), weights=weights.ravel()
    )
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        hist = -kT * np.log(hist)
    hist -= np.nanmin(hist)
    return hist

In [ ]:
def make_3dpmf(x,y,z,weights,kT,xedges,yedges,zedges):
    """Compute  3D PMF (no plot)

    This function histograms the data, weighted by the change of measure, 
    to generate a 3D PMF

    Parameters
    ----------
    x, y, z : ndarray or list/tuple of ndarray
        Coordinates of each frame.
    weights : ndarray or list/tuple of ndarray
        Weight or value of each frame (change of measure)
    kt: float
        Value for Boltzmann constant and temperature (choice of units)
    xedges, yedges, zedges : (nx+1,) or (ny+1,) or (nz+1,) ndarray
        The edges of the PMF 

    Returns
    -------
    hist : (nx, ny, nz) ndarray
        Histogram of the 2D PMF
    """
    x = _flatten(x)
    y = _flatten(y)
    z = _flatten(z)
    weights = _flatten(weights)
    
    hist, _ = np.histogramdd(
        (list(x), list(y), list(z)), weights=weights, bins=(xedges, yedges, zedges)
    )
    
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        hist = -kT * np.log(hist)
    hist -= np.nanmin(hist)

    return hist

In [ ]:
def plot_avg(x, y, v, weights, xedges, yedges, label=None, ax=None,
                title=None, xlabel='x', ylabel='y',
                cmap='plasma',q=False, contours=False,clines=np.linspace(0,50,51),**kwargs):
    """Compute and plot a 2D average

    This function histograms the data, weighted by the change of measure, 
    to generate an average

    Parameters
    ----------
    x, y : ndarray or list/tuple of ndarray
        Coordinates of each frame.
    v : ndarray or list/tuple of ndarray
        Value to be averaged for each frame
    weights : ndarray or list/tuple of ndarray
        Weight or value of each frame (change of measure)
    xedges, yedges : (nx+1,ny+1) ndarray
        The edges of the PMF 
    plotting parameters

    Returns
    -------
    hist : (nx, ny) ndarray
        Histogram of the 2D PMF
    """
    # bin values
    numer, _, _ = np.histogram2d(
        x.ravel(), y.ravel(), bins=(xedges, yedges), weights=v.ravel() * weights.ravel()
    )
    denom, _, _ = np.histogram2d(
        x.ravel(), y.ravel(), bins=(xedges, yedges), weights=weights.ravel()
    )
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        hist = numer / denom
        denom = -kT * np.log(denom)
    denom -= np.nanmin(denom)
    # plot
    if ax is None:
        ax = plt.gca()
    fig = ax.get_figure()
    pcm = ax.pcolormesh(xedges, yedges, hist.T, shading="flat", cmap=cmap,**kwargs)
    ax.set_xlim(xedges[0], xedges[-1])
    ax.set_ylim(yedges[0], yedges[-1])
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    cbar = fig.colorbar(pcm, ax=ax)
    if label is not None:
        cbar.set_label(label, rotation=-90, va="baseline")
    #Draw contours of the PMF if desired
    if contours:
        centerx =(xedges[1:]+xedges[:-1])/2
        centery =(yedges[1:]+yedges[:-1])/2
        if q:
            ax.contour(centerx, centery, hist.T, levels=clines, colors='black', linestyles='solid', linewidths=1.0)
            ax.contour(centerx, centery, hist.T, levels=[0.5], colors='purple', linestyles='solid', linewidths=2.5)
        else:
            ax.contour(centerx, centery, denom.T, levels=clines, colors='black', linestyles='solid', linewidths=0.7)
    return hist

In [ ]:
def make_avg(x, y, v, weights, xedges, yedges):
    """Compute a 2D average (no plot)

    This function histograms the data, weighted by the change of measure, 
    to generate an average

    Parameters
    ----------
    x, y : ndarray or list/tuple of ndarray
        Coordinates of each frame.
    v : ndarray or list/tuple of ndarray
        Value to be averaged for each frame
    weights : ndarray or list/tuple of ndarray
        Weight or value of each frame (change of measure)
    xedges, yedges : (nx+1,ny+1) ndarray
        The edges of the PMF 

    Returns
    -------
    hist : (nx, ny) ndarray
        Histogram of the 2D PMF
    """
    # bin values
    numer, _, _ = np.histogram2d(
        x.ravel(), y.ravel(), bins=(xedges, yedges), weights=v.ravel() * weights.ravel()
    )
    denom, _, _ = np.histogram2d(
        x.ravel(), y.ravel(), bins=(xedges, yedges), weights=weights.ravel()
    )
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        hist = numer / denom
        denom = -kT * np.log(denom)
    denom -= np.nanmin(denom)
    return hist

In [ ]:
import numpy as np
import scipy.ndimage


def kdesum2d(
    x,
    y,
    w1,
    w2,
    *,
    xmin=None,
    xmax=None,
    ymin=None,
    ymax=None,
    xstd=None,
    ystd=None,
    nx=100,
    ny=100,
    cut=4.0,
    plot=False,
    axis=None,
    xlabel=None,
    ylabel=None,
    title=None,
    norm=False,
    pcolor=False,
    cbar=True,
    **kwargs,
):
    """Compute a 2D kernel density estimate.

    This function histograms the data, then uses a Gaussian filter to
    approximate a kernel density estimate with a Gaussian kernel.

    Parameters
    ----------
    x, y : ndarray or list/tuple of ndarray
        Coordinates of each frame.
    w : ndarray or list/tuple of ndarray
        Weight or value of each frame. The output is the sum of these
        values in each bin, after smoothing.
    xmin, xmax, ymin, ymax : float, optional
        Limits of kernel density estimate. If None, takes the min/max
        of the data along the coordinate.
    xstd, ystd : float, optional
        Standard deviation of the Gaussian filter. If None, these are
        set to (xmax - xmin) / nx and (ymax - ymin) / ny, respectively.
        Increase this to smooth the results more.
    nx, ny : int, optional
        Number of bins in each dimension. This should be set as high as
        reasonable, since xstd/ystd takes care of the smoothing.
    cut : float, optional
        Number of standard deviations at which to truncate the Gaussian
        filter. The default, 4, usually doesn't need to be changed.

    Returns
    -------
    kde : (nx, ny) ndarray
        Kernel density estimate, given as bins.
    xedges : (nx+1,) ndarray
        Bin edges along the x dimension.
    yedges : (ny+1,) ndarray
        Bin edges along the y dimension.

    """

    # flatten input to 1D arrays
    x = _flatten(x)
    y = _flatten(y)
    w1 = _flatten(w1)
    w2 = _flatten(w2)

    # limits
    _xmin = np.min(x)
    _xmax = np.max(x)
    _ymin = np.min(y)
    _ymax = np.max(y)
    if xmin is None:
        xmin = _xmin
    if xmax is None:
        xmax = _xmax
    if ymin is None:
        ymin = _ymin
    if ymax is None:
        ymax = _ymax

    # separation between grid points
    xsep = (xmax - xmin) / nx
    ysep = (ymax - ymin) / ny

    # number of grid points to pad the boundaries,
    # since the Gaussian filter extends beyond the boundaries
    # usually overestimates the padding, but whatever
    ax = max(0, int(np.ceil((xmin - _xmin) / xsep + 1e-6)))
    bx = max(0, int(np.ceil((_xmax - xmax) / xsep + 1e-6)))
    ay = max(0, int(np.ceil((ymin - _ymin) / ysep + 1e-6)))
    by = max(0, int(np.ceil((_ymax - ymax) / ysep + 1e-6)))

    # output bin edges
    xedges = np.linspace(xmin, xmax, nx + 1)
    yedges = np.linspace(ymin, ymax, ny + 1)

    # bin edges, with the added padding
    xedges_padded = np.concatenate(
        [
            xmin + xsep * np.arange(-ax, 0),
            xedges,
            xmax + xsep * np.arange(1, bx + 1),
        ]
    )
    yedges_padded = np.concatenate(
        [
            ymin + ysep * np.arange(-ay, 0),
            yedges,
            ymax + ysep * np.arange(1, by + 1),
        ]
    )
    assert np.allclose(xedges_padded[1:] - xedges_padded[:-1], xsep)
    assert np.allclose(yedges_padded[1:] - yedges_padded[:-1], ysep)
    assert xedges_padded[0] <= _xmin and _xmax <= xedges_padded[-1]
    assert yedges_padded[0] <= _ymin and _ymax <= yedges_padded[-1]

    # construct 2D histogram on padded edges
    hist_padded1, _, _ = np.histogram2d(
        x, y, weights=w1, bins=(xedges_padded, yedges_padded)
    )
    
     # construct 2D histogram on padded edges
    hist_padded2, _, _ = np.histogram2d(
        x, y, weights=w2, bins=(xedges_padded, yedges_padded)
    )

    # Gaussian kernel parameters
    if xstd is None:
        xstd = xsep
    if ystd is None:
        ystd = ysep

    # apply Gaussian filter to histogram
    kde_padded1 = scipy.ndimage.gaussian_filter(
        hist_padded1,
        sigma=(xstd / xsep, ystd / ysep),  # in units of grid points
        mode="constant",
        truncate=cut,
    )
    
    kde_padded2 = scipy.ndimage.gaussian_filter(
        hist_padded2,
        sigma=(xstd / xsep, ystd / ysep),  # in units of grid points
        mode="constant",
        truncate=cut,
    )


    # remove the padding
    assert ax + nx + bx == kde_padded1.shape[0]
    assert ay + ny + by == kde_padded1.shape[1]
    kde1 = kde_padded1[ax : ax + nx, ay : ay + ny]
    assert ax + nx + bx == kde_padded2.shape[0]
    assert ay + ny + by == kde_padded2.shape[1]
    kde2 = kde_padded2[ax : ax + nx, ay : ay + ny]

    if plot:
        xc = 0.5 * (xedges[1:] + xedges[:-1])
        yc = 0.5 * (yedges[1:] + yedges[:-1])
        vx=kde1
        vy=kde2
            # split direction and magnitude
        v = np.sqrt(vx ** 2 + vy ** 2)
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            if norm:
                ux = vx / v
                uy = vy / v
            else:
                ux = vx
                uy = vy
            logv = np.log(v)
        ux[v == 0.0] = 0.0
        uy[v == 0.0] = 0.0

        cmap='cividis'
        # plot
        if axis is None:
            axis = plt.gca()
        fig = axis.get_figure()
        if pcolor:
            cmap='coolwarm'
            vmax = np.max(ux)
            pcm = axis.pcolormesh(xedges, yedges, ux.T, shading="flat", cmap=cmap,**kwargs,vmin=-1*vmax,vmax=vmax)
            cbar = fig.colorbar(pcm, ax=axis, format='%.1e')
        else:
            pcm = axis.quiver(
                xc, yc, ux.T, uy.T, v.T, angles="xy", pivot="middle", cmap=cmap,width = 0.0055, edgecolor='black', **kwargs
            )
            if cbar:
                cbar = fig.colorbar(pcm, ax=axis, format='%.1e')
                #cbar.set_label(None, rotation=-90, va="baseline")
        axis.set_xlim(xedges[0], xedges[-1])
        axis.set_ylim(yedges[0], yedges[-1])
        axis.set_xlabel(xlabel)
        axis.set_ylabel(ylabel)
        axis.set_title(title)
        
        
    return kde1, kde2, xedges, yedges


def _flatten(a):
    if isinstance(a, np.ndarray):
        # avoid creating a new array (and using twice the memory)
        return np.ravel(a)
    else:
        return np.ravel(np.concatenate(a))

In [ ]:
import numpy as np
import scipy.ndimage


def kdesum2d1(
    x,
    y,
    w1,
    w2,
    *,
    xmin=None,
    xmax=None,
    ymin=None,
    ymax=None,
    xstd=None,
    ystd=None,
    nx=100,
    ny=100,
    cut=4.0,
    plot=False,
    axis=None,
    xlabel=None,
    ylabel=None,
    title=None,
    norm=False,
    pcolor=False,
    cbar=True,
    label=None,
    **kwargs,
):
    """Compute a 2D kernel density estimate - with adjusted plotting parameters.

    This function histograms the data, then uses a Gaussian filter to
    approximate a kernel density estimate with a Gaussian kernel.

    Parameters
    ----------
    x, y : ndarray or list/tuple of ndarray
        Coordinates of each frame.
    w : ndarray or list/tuple of ndarray
        Weight or value of each frame. The output is the sum of these
        values in each bin, after smoothing.
    xmin, xmax, ymin, ymax : float, optional
        Limits of kernel density estimate. If None, takes the min/max
        of the data along the coordinate.
    xstd, ystd : float, optional
        Standard deviation of the Gaussian filter. If None, these are
        set to (xmax - xmin) / nx and (ymax - ymin) / ny, respectively.
        Increase this to smooth the results more.
    nx, ny : int, optional
        Number of bins in each dimension. This should be set as high as
        reasonable, since xstd/ystd takes care of the smoothing.
    cut : float, optional
        Number of standard deviations at which to truncate the Gaussian
        filter. The default, 4, usually doesn't need to be changed.

    Returns
    -------
    kde : (nx, ny) ndarray
        Kernel density estimate, given as bins.
    xedges : (nx+1,) ndarray
        Bin edges along the x dimension.
    yedges : (ny+1,) ndarray
        Bin edges along the y dimension.

    """

    # flatten input to 1D arrays
    x = _flatten(x)
    y = _flatten(y)
    w1 = _flatten(w1)
    w2 = _flatten(w2)

    # limits
    _xmin = np.min(x)
    _xmax = np.max(x)
    _ymin = np.min(y)
    _ymax = np.max(y)
    if xmin is None:
        xmin = _xmin
    if xmax is None:
        xmax = _xmax
    if ymin is None:
        ymin = _ymin
    if ymax is None:
        ymax = _ymax

    # separation between grid points
    xsep = (xmax - xmin) / nx
    ysep = (ymax - ymin) / ny

    # number of grid points to pad the boundaries,
    # since the Gaussian filter extends beyond the boundaries
    # usually overestimates the padding, but whatever
    ax = max(0, int(np.ceil((xmin - _xmin) / xsep + 1e-6)))
    bx = max(0, int(np.ceil((_xmax - xmax) / xsep + 1e-6)))
    ay = max(0, int(np.ceil((ymin - _ymin) / ysep + 1e-6)))
    by = max(0, int(np.ceil((_ymax - ymax) / ysep + 1e-6)))

    # output bin edges
    xedges = np.linspace(xmin, xmax, nx + 1)
    yedges = np.linspace(ymin, ymax, ny + 1)

    # bin edges, with the added padding
    xedges_padded = np.concatenate(
        [
            xmin + xsep * np.arange(-ax, 0),
            xedges,
            xmax + xsep * np.arange(1, bx + 1),
        ]
    )
    yedges_padded = np.concatenate(
        [
            ymin + ysep * np.arange(-ay, 0),
            yedges,
            ymax + ysep * np.arange(1, by + 1),
        ]
    )
    assert np.allclose(xedges_padded[1:] - xedges_padded[:-1], xsep)
    assert np.allclose(yedges_padded[1:] - yedges_padded[:-1], ysep)
    assert xedges_padded[0] <= _xmin and _xmax <= xedges_padded[-1]
    assert yedges_padded[0] <= _ymin and _ymax <= yedges_padded[-1]

    # construct 2D histogram on padded edges
    hist_padded1, _, _ = np.histogram2d(
        x, y, weights=w1, bins=(xedges_padded, yedges_padded)
    )
    
     # construct 2D histogram on padded edges
    hist_padded2, _, _ = np.histogram2d(
        x, y, weights=w2, bins=(xedges_padded, yedges_padded)
    )

    # Gaussian kernel parameters
    if xstd is None:
        xstd = xsep
    if ystd is None:
        ystd = ysep

    # apply Gaussian filter to histogram
    kde_padded1 = scipy.ndimage.gaussian_filter(
        hist_padded1,
        sigma=(xstd / xsep, ystd / ysep),  # in units of grid points
        mode="constant",
        truncate=cut,
    )
    
    kde_padded2 = scipy.ndimage.gaussian_filter(
        hist_padded2,
        sigma=(xstd / xsep, ystd / ysep),  # in units of grid points
        mode="constant",
        truncate=cut,
    )


    # remove the padding
    assert ax + nx + bx == kde_padded1.shape[0]
    assert ay + ny + by == kde_padded1.shape[1]
    kde1 = kde_padded1[ax : ax + nx, ay : ay + ny]
    assert ax + nx + bx == kde_padded2.shape[0]
    assert ay + ny + by == kde_padded2.shape[1]
    kde2 = kde_padded2[ax : ax + nx, ay : ay + ny]

    if plot:
        xc = 0.5 * (xedges[1:] + xedges[:-1])
        yc = 0.5 * (yedges[1:] + yedges[:-1])
        vx=kde1
        vy=kde2
            # split direction and magnitude
        v = np.sqrt(vx ** 2 + vy ** 2)
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            if norm:
                ux = vx / v
                uy = vy / v
            else:
                ux = vx
                uy = vy
            logv = np.log(v)
        ux[v == 0.0] = 0.0
        uy[v == 0.0] = 0.0

        cmap='cividis'
        # plot
        if axis is None:
            axis = plt.gca()
        fig = axis.get_figure()
        if pcolor:
            cmap='coolwarm'
            vmax = np.max(ux)
            pcm = axis.pcolormesh(xedges, yedges, ux.T, shading="flat", cmap=cmap,**kwargs,vmin=-1*vmax,vmax=vmax)
            cbar = fig.colorbar(pcm, ax=axis, format='%.1e')
        else:
            pcm = axis.quiver(
                xc, yc, ux.T, uy.T, v.T, angles="xy", pivot="middle", cmap=cmap,width = 0.0055, edgecolor='black', **kwargs
            )
            if cbar:
                cbformat = mpl.ticker.ScalarFormatter()   # create the formatter
                cbformat.set_powerlimits((-7,-5)) 
                cbar = fig.colorbar(pcm, ax=axis, format=cbformat)
                cbar.set_label(label, rotation=-90, va="baseline", fontsize=16)
                cbar.ax.tick_params(width=1.5, labelsize=14)
        axis.set_xlim(xedges[0], xedges[-1])
        axis.set_ylim(yedges[0], yedges[-1])
        axis.set_xlabel(xlabel)
        axis.set_ylabel(ylabel)
        axis.set_title(title)
        
        
    return kde1, kde2, xedges, yedges


def _flatten(a):
    if isinstance(a, np.ndarray):
        # avoid creating a new array (and using twice the memory)
        return np.ravel(a)
    else:
        return np.ravel(np.concatenate(a))

In [ ]:
def kdesum3d(
    x,
    y,
    z,
    w1,
    w2,
    w3,
    *,
    xmin=0,
    xmax=100,
    ymin=0,
    ymax=100,
    zmin=0,
    zmax=100,
    xstd=None,
    ystd=None,
    zstd=None,
    nx=50,
    ny=50,
    nz=50,
    cut=4.0,
    plot=False,
    axis=None,
    xlabel=None,
    ylabel=None,
    zlabel=None,
    title=None,
    norm=False,
    **kwargs,
):
    """Compute a 3D kernel density estimate.

    This function histograms the data, then uses a Gaussian filter to
    approximate a kernel density estimate with a Gaussian kernel.

    Parameters
    ----------
    x, y, z : ndarray or list/tuple of ndarray
        Coordinates of each frame.
    w1, w2 : ndarray or list/tuple of ndarray
        Weight or value of each frame. The output is the sum of these
        values in each bin, after smoothing.
    xmin, xmax, ymin, ymax, zmin, zmax : float, optional
        Limits of kernel density estimate. If None, takes the min/max
        of the data along the coordinate.
    xstd, ystd, zstd : float, optional
        Standard deviation of the Gaussian filter. If None, these are
        set to (xmax - xmin) / nx and (ymax - ymin) / ny, respectively.
        Increase this to smooth the results more.
    nx, ny, nz : int, optional
        Number of bins in each dimension. This should be set as high as
        reasonable, since xstd/ystd takes care of the smoothing.
    cut : float, optional
        Number of standard deviations at which to truncate the Gaussian
        filter. The default, 4, usually doesn't need to be changed.

    Returns
    -------
    kde : (nx, ny) ndarray
        Kernel density estimate, given as bins.
    xedges : (nx+1,) ndarray
        Bin edges along the x dimension.
    yedges : (ny+1,) ndarray
        Bin edges along the y dimension.
    zedges: (nz+1,) ndarray
        Bin edges along the z dimension.

    """

    # flatten input to 1D arrays
    x = _flatten(x)
    y = _flatten(y)
    z = _flatten(z)
    w1 = _flatten(w1)
    w2 = _flatten(w2)
    w3 = _flatten(w3)

    # limits
    _xmin = np.min(x)
    _xmax = np.max(x)
    _ymin = np.min(y)
    _ymax = np.max(y)
    _zmin = np.min(z)
    _zmax = np.max(z)
    if xmin is None:
        xmin = _xmin
    if xmax is None:
        xmax = _xmax
    if ymin is None:
        ymin = _ymin
    if ymax is None:
        ymax = _ymax
    if zmin is None:
        zmin = _zmin
    if zmax is None:
        zmax = _zmax

    # separation between grid points
    xsep = (xmax - xmin) / nx
    ysep = (ymax - ymin) / ny
    zsep = (zmax - zmin) / nz

    # number of grid points to pad the boundaries,
    # since the Gaussian filter extends beyond the boundaries
    # usually overestimates the padding, but whatever
    ax = max(0, int(np.ceil((xmin - _xmin) / xsep + 1e-6)))
    bx = max(0, int(np.ceil((_xmax - xmax) / xsep + 1e-6)))
    ay = max(0, int(np.ceil((ymin - _ymin) / ysep + 1e-6)))
    by = max(0, int(np.ceil((_ymax - ymax) / ysep + 1e-6)))
    az = max(0, int(np.ceil((zmin - _zmin) / zsep + 1e-6)))
    bz = max(0, int(np.ceil((_zmax - zmax) / zsep + 1e-6)))

    # output bin edges
    xedges = np.linspace(xmin, xmax, nx + 1)
    yedges = np.linspace(ymin, ymax, ny + 1)
    zedges = np.linspace(zmin, zmax, nz + 1)

    # bin edges, with the added padding
    xedges_padded = np.concatenate(
        [
            xmin + xsep * np.arange(-ax, 0),
            xedges,
            xmax + xsep * np.arange(1, bx + 1),
        ]
    )
    yedges_padded = np.concatenate(
        [
            ymin + ysep * np.arange(-ay, 0),
            yedges,
            ymax + ysep * np.arange(1, by + 1),
        ]
    )
    zedges_padded = np.concatenate(
        [
            zmin + zsep * np.arange(-az, 0),
            zedges,
            zmax + zsep * np.arange(1, bz + 1),
        ]
    )
    assert np.allclose(xedges_padded[1:] - xedges_padded[:-1], xsep)
    assert np.allclose(yedges_padded[1:] - yedges_padded[:-1], ysep)
    assert np.allclose(zedges_padded[1:] - zedges_padded[:-1], zsep)
    assert xedges_padded[0] <= _xmin and _xmax <= xedges_padded[-1]
    assert yedges_padded[0] <= _ymin and _ymax <= yedges_padded[-1]
    assert zedges_padded[0] <= _zmin and _zmax <= zedges_padded[-1]

    # construct 3D histogram on padded edges
    hist_padded1, _ = np.histogramdd(
        (list(x), list(y), list(z)), weights=w1, bins=(xedges_padded, yedges_padded, zedges_padded)
    )
    
     # construct 3D histogram on padded edges
    hist_padded2, _ = np.histogramdd(
        (list(x), list(y), list(z)), weights=w2, bins=(xedges_padded, yedges_padded, zedges_padded)
    )
    
    # construct 3D histogram on padded edges
    hist_padded3, _ = np.histogramdd(
        (list(x), list(y), list(z)), weights=w3, bins=(xedges_padded, yedges_padded, zedges_padded)
    )


    # Gaussian kernel parameters
    if xstd is None:
        xstd = xsep
    if ystd is None:
        ystd = ysep
    if zstd is None:
        zstd = zsep

    # apply Gaussian filter to histogram
    kde_padded1 = scipy.ndimage.gaussian_filter(
        hist_padded1,
        sigma=(xstd / xsep, ystd / ysep, zstd / zsep),  # in units of grid points
        mode="constant",
        truncate=cut,
    )
    
    kde_padded2 = scipy.ndimage.gaussian_filter(
        hist_padded2,
        sigma=(xstd / xsep, ystd / ysep, zstd / zsep),  # in units of grid points
        mode="constant",
        truncate=cut,
    )
    kde_padded3 = scipy.ndimage.gaussian_filter(
        hist_padded3,
        sigma=(xstd / xsep, ystd / ysep, zstd / zsep),  # in units of grid points
        mode="constant",
        truncate=cut,
    )



    # remove the padding
    assert ax + nx + bx == kde_padded1.shape[0]
    assert ay + ny + by == kde_padded1.shape[1]
    assert az + nz + bz == kde_padded1.shape[2]
    kde1 = kde_padded1[ax : ax + nx, ay : ay + ny, az : az + nz]
    assert ax + nx + bx == kde_padded2.shape[0]
    assert ay + ny + by == kde_padded2.shape[1]
    assert az + nz + bz == kde_padded2.shape[2]
    kde2 = kde_padded2[ax : ax + nx, ay : ay + ny, az : az + nz]
    assert ax + nx + bx == kde_padded3.shape[0]
    assert ay + ny + by == kde_padded3.shape[1]
    assert az + nz + bz == kde_padded3.shape[2]
    kde3 = kde_padded3[ax : ax + nx, ay : ay + ny, az : az + nz]

    if plot:
        xc = 0.5 * (xedges[1:] + xedges[:-1])
        yc = 0.5 * (yedges[1:] + yedges[:-1])
        zc = 0.5 * (zedges[1:] + zedges[:-1])
        vx=kde1
        vy=kde2
        vz=kde3
        # split direction and magnitude
        v = np.sqrt(vx ** 2 + vy ** 2 + vz ** 2)
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            if norm:
                ux = vx / v
                uy = vy / v
                uz = vz / v
            else:
                ux = vx
                uy = vy
                uz = vz
            logv = np.log(v)
        ux[v == 0.0] = 0.0
        uy[v == 0.0] = 0.0
        uz[v == 0.0] = 0.0

        cmap='cividis'
        # plot
        if axis is None:
            axis = plt.gca()
        fig = axis.get_figure()
        start=None
        stop=None
        Skip=1
        
        X,Y,Z=np.meshgrid(xc,yc,zc)
        y = X[0::Skip,0::Skip,0::Skip].reshape(np.product(X[0::Skip,0::Skip,0::Skip].shape))
        x = Y[0::Skip,0::Skip,0::Skip].reshape(np.product(Y[0::Skip,0::Skip,0::Skip].shape))
        z = Z[0::Skip,0::Skip,0::Skip].reshape(np.product(Z[0::Skip,0::Skip,0::Skip].shape))
        
        ux = _flatten(ux)
        uy = _flatten(uy)
        uz = _flatten(uz)
        v = _flatten(v)
    
        for x1,y1,z1,u1,v1,w1,l in zip(x[start:stop],y[start:stop],z[start:stop],ux[start:stop],uy[start:stop],uz[start:stop],v[start:stop]):
            axis.quiver(x1, y1, z1, u1, v1, w1, pivot = 'middle', length=min(8.0,l*1000000), normalize=True, arrow_length_ratio=0.5)
            
        axis.set_xlim(xedges[0], xedges[-1])
        axis.set_ylim(yedges[0], yedges[-1])
        axis.set_zlim(zedges[0], zedges[-1])
        axis.set_xlabel(xlabel)
        axis.set_ylabel(ylabel)
        axis.set_zlabel(ylabel)
        axis.set_title(title)
        
        
    return kde1, kde2, kde3, xedges, yedges, zedges


In [ ]:
def plot_pmf1(x,y,weights,kT,xedges,yedges,
                title=None,xlabel='x',ylabel='y',label=None,ax=None,vmin=None,vmax=None,
                colorbar=True,cmap='cividis',contours=True,pcolor=True,clines=np.linspace(0,50,51),**kwargs):
    """Compute and plot a 2D PMF - with adjusted plotting parameters

    This function histograms the data, weighted by the change of measure, 
    to generate a PMF

    Parameters
    ----------
    x, y : ndarray or list/tuple of ndarray
        Coordinates of each frame.
    weights : ndarray or list/tuple of ndarray
        Weight or value of each frame (change of measure)
    kt: float
        Value for Boltzmann constant and temperature (choice of units)
    xedges, yedges : (nx+1,ny+1) ndarray
        The edges of the PMF 
    plotting parameters

    Returns
    -------
    hist : (nx, ny) ndarray
        Histogram of the 2D PMF
    """
    # bin values
    hist, _, _ = np.histogram2d(
        x.ravel(), y.ravel(), bins=(xedges, yedges), weights=weights.ravel()
    )
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        hist = -kT * np.log(hist)
    hist -= np.nanmin(hist)
    if vmin is not None or vmax is not None:
        if vmin is not None:
            hist[hist < vmin] = np.nan
        if vmax is not None:
            hist[hist > vmax] = np.nan
    # plot the PMF in bins
    if ax is None:
        ax = plt.gca()
    fig = ax.get_figure()
    if pcolor:
        pcm = ax.pcolormesh(xedges, yedges, hist.T, shading="flat", cmap=cmap, **kwargs)
        if colorbar:
            cbar = fig.colorbar(pcm, ax=ax)
        if label is not None:
            cbar.set_label(label, rotation=-90, va="baseline", fontsize=16)
            cbar.ax.tick_params(width=1.5, labelsize=14)
    ax.set_xlim(xedges[0], xedges[-1])
    ax.set_ylim(yedges[0], yedges[-1])
    #ax.set_aspect("equal")
    ax.set_xlabel(xlabel, fontsize=16)
    ax.set_ylabel(ylabel, fontsize=16)
    ax.set_title(title, fontsize=16)
    ax.tick_params(width=1.5, labelsize=14)
    #Draw contours if desired
    if contours:
        centerx =(xedges[1:]+xedges[:-1])/2
        centery =(yedges[1:]+yedges[:-1])/2
        if pcolor:
            ax.contour(centerx, centery, hist.T, levels=clines, colors='black', linestyles='solid', linewidths=1)
        else: 
            ax.contour(centerx, centery, hist.T, levels=clines, colors='black', linestyles='solid', linewidths=1)
    return hist

In [ ]:
def plot_avg1(x, y, v, weights, xedges, yedges, label=None, ax=None,
                title=None, xlabel='x', ylabel='y',
                cmap='plasma',q=False, contours=False,clines=np.linspace(0,50,51),**kwargs):
    """Compute and plot a 2D average - with adjusted plotting parameters

    This function histograms the data, weighted by the change of measure, 
    to generate an average

    Parameters
    ----------
    x, y : ndarray or list/tuple of ndarray
        Coordinates of each frame.
    v : ndarray or list/tuple of ndarray
        Value to be averaged for each frame
    weights : ndarray or list/tuple of ndarray
        Weight or value of each frame (change of measure)
    xedges, yedges : (nx+1,ny+1) ndarray
        The edges of the PMF 
    plotting parameters

    Returns
    -------
    hist : (nx, ny) ndarray
        Histogram of the 2D PMF
    """
    # bin values
    numer, _, _ = np.histogram2d(
        x.ravel(), y.ravel(), bins=(xedges, yedges), weights=v.ravel() * weights.ravel()
    )
    denom, _, _ = np.histogram2d(
        x.ravel(), y.ravel(), bins=(xedges, yedges), weights=weights.ravel()
    )
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        hist = numer / denom
        denom = -kT * np.log(denom)
    denom -= np.nanmin(denom)
    # plot
    if ax is None:
        ax = plt.gca()
    fig = ax.get_figure()
    pcm = ax.pcolormesh(xedges, yedges, hist.T, shading="flat", cmap=cmap,**kwargs)
    ax.set_xlim(xedges[0], xedges[-1])
    ax.set_ylim(yedges[0], yedges[-1])
    #ax.set_aspect("equal")
    ax.set_xlabel(xlabel, fontsize=16)
    ax.set_ylabel(ylabel, fontsize=16)
    ax.set_title(title, fontsize=16)
    ax.tick_params(width=1.5, labelsize=14)
    cbar = fig.colorbar(pcm, ax=ax)
    if label is not None:
            cbar.set_label(label, rotation=-90, va="baseline", fontsize=16)
            cbar.ax.tick_params(width=1.5, labelsize=14)
    #Draw contours of the PMF if desired
    if contours:
        centerx =(xedges[1:]+xedges[:-1])/2
        centery =(yedges[1:]+yedges[:-1])/2
        if q:
            ax.contour(centerx, centery, hist.T, levels=clines, colors='black', linestyles='solid', linewidths=1.0)
            ax.contour(centerx, centery, hist.T, levels=[0.5], colors='purple', linestyles='solid', linewidths=2.5)
        else:
            ax.contour(centerx, centery, hist.T, levels=clines, colors='black', linestyles='solid', linewidths=1.0)
    return hist

# Load features from coordinates

Use PyEMMA to import your desired raw features - pairwise distances, dihedral angles, etc. 

In [ ]:
feat3 = pyemma.coordinates.featurizer('PATH_TO_PDB')

# Add atom ids for input features
cas3 = (4761, 4764, 163, 182, 239, 564, 583, 616, 1102, 1122, 1169, 1186, 2436, 2470, 2481, 2500, 2510, 2521, 2540, 2578, 2597, 2805, 2822, 2872)
gks3 = (4761, 4764, 569, 1176, 2489, 2526)

#Add the distances between binding pocket's alpha carbons and IPH1 (protein-protein and protein-ligand)
feat3.add_distances(cas3)
#Distances between gatekeeping gamma carbons and IPH1 (protein-protein and protein-ligand)
feat3.add_distances(gks3)
feat3.dimension()

In [ ]:
#Load in that data from all of your trajectories
dfiles = sorted(glob.glob('PATH_TO_ALL_DATA_FILES'))
raw_feat = pyemma.coordinates.load(dfiles_f, features=feat3)

Load any supplementary data (CVs)

In [ ]:
ccvfiles = sorted(glob.glob('PATH_TO_CVs'))

supp_ctrajs = []
for ccvfile in ccvfiles:
    cv = np.loadtxt(ccvfile)
    supp_ctrajs.append(cv)
supp_CCV_arr = np.vstack(supp_ctrajs)


# Define sets A and B for DGA analysis 
In this case, we are defining our free and bound states. Definitions are found in the paper. 
InD means all those in the domain - namely, not in either A or B

In [ ]:
cvfiles = sorted(glob.glob('PATH_TO_CVFILES'))

ahcvfiles = sorted(glob.glob('PATH_TO_HBOND_FILES'))

cv_trajs = []
InD = []
InBound = []
InFree = []

#Define the parameters for each set
lower_cutoff=5
upper_cutoff=540
n_stdevs = 2
abr_c = np.asarray((0.087238, 52.503, 31.924))
abr_r = np.asarray((0.022239, 5.023, 5.264)) * n_stdevs

print(abr_c, abr_r)

#Assemble the cv trajs, as well as the in/out of domain matrices

#Define unbound as less than 10 contacts with the protein, and in the sphere centered at (abr_c)
for ahcvfile, cvfile, rcvfile in zip(ahcvfiles, cvfiles, rcvfiles):
    cv = np.loadtxt(cvfile)[::2,:]
    ahcv = np.loadtxt(ahcvfile)[::2]
    rcv = np.loadtxt(rcvfile, usecols=(1))[:]
    cv_trajs.append(cv)
    if len(cv) != 4001: print(cvfile, len(cv))
    if len(ahcv) != 4001: print(ahcvfile, len(ahcv))
    if len(rcv) != 4001: print(rcvfile, len(rcv))
    in_bound = np.zeros(len(cv))
    in_free = np.zeros(len(cv))
    ind = np.ones(len(cv))
    point_r = np.sum(((cv[:,3:6] - abr_c) / abr_r)**2, axis=1)
    in_bound[np.logical_and(ahcv > 1.1, np.logical_and(point_r[:] <= 1, cv[:,6] > upper_cutoff))] = 1
    in_bound = in_bound.astype('int')
    in_free[np.logical_and(cv[:,6] < lower_cutoff/2, np.logical_and(cv[:,1] < lower_cutoff, cv[:,2] < lower_cutoff))] = 1
    in_free = in_free.astype('int')
    ind = (ind - in_free - in_bound).astype('int')
    
    InD.append(ind)
    InBound.append(in_bound)
    InFree.append(in_free)

InD_arr = np.reshape(np.hstack(InD), (-1,1))
InBound_arr = np.reshape(np.hstack(InBound), (-1,1))
InFree_arr = np.reshape(np.hstack(InFree), (-1,1))

CV_arr = np.vstack(cv_trajs)

Bound_cv = CV_arr[(InBound_arr==1).flatten()]
Free_cv = CV_arr[(InFree_arr==1).flatten()]
D_cv = CV_arr[(InD_arr==1).flatten()]

# Calculate the distances to the bound/free states

These will be used to calculate the smoothing and guess functions

In [ ]:
d_bound = []
counter=0.0
tot = len(RawFeat_arr)
skip = 3

def reduce_func(D_chunk, start):
    min2 = np.min(D_chunk,axis=1)
    return min2

for chunk in skl.pairwise_distances_chunked(RawFeat_arr[:,:], Bound_arr[::skip,:], n_jobs=-1, reduce_func=reduce_func):
    d_bound.append(chunk)
    counter+=float(len(chunk))
    perc = counter / tot * 100
    print(str(perc) + '% Completed')

d_bounds = np.reshape(np.concatenate(d_bound), (-1,1))
d_bounds[(InBound_arr==1).flatten()]=0


In [ ]:
d_free = []
counter=0.0
tot = len(RawFeat_arr)
def reduce_func(D_chunk, start):
    min2 = np.min(D_chunk,axis=1)
    return min2

for chunk in skl.pairwise_distances_chunked(RawFeat_arr[:,:], Free_arr[:,:], n_jobs=-1, reduce_func=reduce_func):
    d_free.append(chunk)
    counter+=float(len(chunk))
    perc = counter / tot * 100
    print(str(perc) + '% Completed')

d_frees = np.reshape(np.concatenate(d_free), (-1,1))
d_frees[(InFree_arr==1).flatten()]=0

# Calculate the smoothing and guess functions, build the basis
Smooth the raw features earlier generated with PyEMMA

In [ ]:
smooth = (d_frees * d_bounds) / (d_frees + d_bounds)**2
guess_bound = (d_frees)**2 / (d_frees**2 + d_bounds**2)
guess_free = (d_bounds)**2 / (d_frees**2 + d_bounds**2)

smooth[(InD_arr==0).flatten()]=0
guess_bound[(InBound_arr==1).flatten()]=1
guess_bound[(InFree_arr==1).flatten()]=0
guess_free[(InFree_arr==1).flatten()]=1
guess_free[(InBound_arr==1).flatten()]=0

In [ ]:
#First, Smooth after adding a constant function
const = np.ones((len(RawFeat_arr),1))
BasisL_arr = np.hstack((const,RawFeat_arr))
basis_smooth = smooth * BasisL_arr

# #Now, whiten
basis_white,L = svd_whiten(basis_smooth,1,frac_retain=1.0)
basis_white[(InD_arr==0).flatten()]=0

In [ ]:
# #Now, whiten the raw data for COM calculation
basis_COM,L = svd_whiten(RawFeat_arr,1,frac_retain=1.0)
basis_COM[(InD_arr==0).flatten()]=0

In [ ]:
#Convert to the correct data format (list of arrays instead one long array)

BasisF_arr = basis_white
BasisC_arr = basis_COM
GuessF_arr = guess_bound
GuessUF_arr = guess_free
#InD already exists as a list of arrays
BasisF= []
BasisC = []
GuessUF= []
GuessF= []

traj_length = len(raw_feat[0])
f_start = 0
for i in range(len(raw_feat)):
    BasisF.append(BasisF_arr[f_start:f_start+traj_length,:])
    BasisC.append(BasisC_arr[f_start:f_start+traj_length,:])
    GuessUF.append(GuessUF_arr[f_start:f_start+traj_length,:].flatten())
    GuessF.append(GuessF_arr[f_start:f_start+traj_length,:].flatten())
    f_start+=traj_length
    

# Start calculating DGA quantities

In [ ]:
#Calculate Change of Measure

C_COMInds = []
for i,LagTime in enumerate(lags[]):
    com=extq.dga.reweight(BasisC,LagTime)
    print(np.min(com), np.max(com), LagTime)
    C_COMInds.append(com)

In [ ]:
#Calculate forward and backward committors for both the binding and unbinding process

C_QIndsUnfold = []
C_QIndsFold = []
C_QbIndsUnfold = []
C_QbIndsFold = []

for i,LagTime in enumerate(lags):
    C_QIndsUnfold.append(extq.dga.forward_committor(BasisF,C_COMInds[i],C_InD,GuessUF,LagTime))
    C_QbIndsUnfold.append(extq.dga.backward_committor(BasisF,C_COMInds[i],C_InD,GuessF,LagTime))
    C_QIndsFold.append(extq.dga.forward_committor(BasisF,C_COMInds[i],C_InD,GuessF,LagTime))
    C_QbIndsFold.append(extq.dga.backward_committor(BasisF,C_COMInds[i],C_InD,GuessUF,LagTime))
    print(LagTime)

In [ ]:
# Explicitly make sure committors obey boundary conditions

C_QIndsUnfold = np.asarray(C_QIndsUnfold)
C_QbIndsUnfold = np.asarray(C_QbIndsUnfold)
C_QIndsFold= np.asarray(C_QIndsFold)
C_QbIndsFold = np.asarray(C_QbIndsFold)

C_QIndsUnfold[C_QIndsUnfold > 1] = 1
C_QIndsUnfold[C_QIndsUnfold < 0] = 0

C_QbIndsUnfold[C_QbIndsUnfold > 1] = 1
C_QbIndsUnfold[C_QbIndsUnfold < 0] = 0

C_QbIndsFold[C_QbIndsFold > 1] = 1
C_QbIndsFold[C_QbIndsFold < 0] = 0

C_QIndsFold[C_QIndsFold > 1] = 1
C_QIndsFold[C_QIndsFold < 0] = 0


In [ ]:
# Calculate rates

urates = []
frates = []
for i, Lag in enumerate(lags):
    urates.append(extq.tpt.rate(C_QIndsUnfold[i],C_QbIndsUnfold[i],C_COMInds[i],C_InD,C_QIndsUnfold[i],Lag))
    frates.append(extq.tpt.rate(C_QIndsFold[i],C_QbIndsFold[i],C_COMInds[i],C_InD,C_QIndsFold[i],Lag))
    print(Lag)

In [ ]:
#Convert to unimolecular rate constants

kF_denoms = []
kU_denoms = []

for i, lag in enumerate(lags):
    trjlen = np.asarray(C_QbIndsFold[i]).shape[1]
    kF_denom = np.sum(np.asarray(C_QbIndsFold[i])[:,:(trjlen-lag)]* 
                      np.asarray(C_COMInds[i])[:,:(trjlen-lag)]) / np.sum(np.asarray(C_COMInds[i])[:,:(trjlen-lag)])
    kU_denom = np.sum(np.asarray(C_QbIndsUnfold[i])[:,:(trjlen-lag)]* 
                      np.asarray(C_COMInds[i])[:,:(trjlen-lag)]) / np.sum(np.asarray(C_COMInds[i])[:,:(trjlen-lag)])
    kF_denoms.append(kF_denom)
    kU_denoms.append(kU_denom)
    print(kF_denom, kU_denom, trjlen)
kF_denoms = np.asarray(kF_denoms)
kU_denoms = np.asarray(kU_denoms)

kF = np.asarray(frates) / kF_denoms
kU = np.asarray(urates) / kU_denoms
K_D = kU / kF

In [ ]:
#Convert to bimolecular rate constants

#Load the distance between phenol and ZN COM
phen_r = np.load('c_extended40_final/phen_r.npy')

kF_denoms = []
kU_denoms = []
bi_frates = []
urate_mods = []

#Diffusion constant in cm^2/s
D=2.6529e-5 + 0.0396e-5

#Radii in nm
b=3.3
c=5.3
phen_r_new = np.reshape(phen_r, C_QbIndsFold[0].shape)

for i, lag in enumerate(lags):
    trjlen = np.asarray(C_QbIndsFold[i]).shape[1]
    in_range = np.logical_and(phen_r_new > b - 0.05, phen_r_new < b + 0.05)
    q_bind = np.mean(C_QbIndsFold[i][in_range])
    f = q_bind / (1.0 - ((b/c)*(1.0-q_bind)))
    bi_frate = 4.0 * np.pi * D * b * 1e-7 * f * 6.022e23 / 1000
    urate_mod = 1.0 - ((b/c)*f)
    print(np.shape(C_QbIndsFold[i][in_range]), np.mean(C_QbIndsFold[i][in_range]), bi_frate)
    kF_denom = np.sum(np.asarray(C_QbIndsFold[i])[:,:(trjlen-lag)]* 
                      np.asarray(C_COMInds[i])[:,:(trjlen-lag)]) / np.sum(np.asarray(C_COMInds[i])[:,:(trjlen-lag)])
    kU_denom = np.sum(np.asarray(C_QbIndsUnfold[i])[:,:(trjlen-lag)]* 
                      np.asarray(C_COMInds[i])[:,:(trjlen-lag)]) / np.sum(np.asarray(C_COMInds[i])[:,:(trjlen-lag)])
    kF_denoms.append(kF_denom)
    kU_denoms.append(kU_denom)
    bi_frates.append(bi_frate)
    urate_mods.append(urate_mod)
    print(kF_denom, kU_denom, trjlen)
kF_denoms = np.asarray(kF_denoms)
kU_denoms = np.asarray(kU_denoms)

kF = np.asarray(bi_frates)
kU = np.asarray(urates) * np.asarray(urate_mods) / (kU_denoms * 10 * 1e-12)
K_D = kU / kF
print(K_D)

In [ ]:
#Calculate reactive currents along any dimension (here we do 4)

JABx = []
JABy = []
JABz = []
qJABz = []
JBAx = []
JBAy = []
JBAz = []
qJBAz = []

cv1 = [col[:,4] for col in cv_trajs]
cv2 = [col[:,5] for col in cv_trajs]
cv3 = [col[:,5] for col in supp_cv_trajs]

for i, (LagTime,com, qf, qb) in enumerate(zip(lags, C_COMInds, C_QIndsUnfold, C_QbIndsUnfold)):
    JABx.append((extq.tpt.current(qf, qb, com, C_InD, cv1, LagTime)))
    JABy.append((extq.tpt.current(qf, qb, com, C_InD, cv2, LagTime)))
    JABz.append((extq.tpt.current(qf, qb, com, C_InD, cv3, LagTime)))
    qJABz.append((extq.tpt.current(qf, qb, com, C_InD, qf, LagTime)))
    print(LagTime)
for i, (LagTime,com, qf, qb) in enumerate(zip(lags, C_COMInds, C_QIndsFold, C_QbIndsFold)):
    JBAx.append((extq.tpt.current(qf, qb, com, C_InD, cv1, LagTime)))
    JBAy.append((extq.tpt.current(qf, qb, com, C_InD, cv2, LagTime)))
    JBAz.append((extq.tpt.current(qf, qb, com, C_InD, cv3, LagTime)))
    qJBAz.append((extq.tpt.current(qf, qb, com, C_InD, qf, LagTime)))
    print(LagTime)

# Now let's plot some representative figures

In [ ]:
kT=1
xlim = np.linspace(0,100,51)
ylim = np.linspace(0,100,51)
xlim_flux = np.linspace(0,100,26)
ylim_flux = np.linspace(0,100,26)
zlim_flux = np.linspace(0,100,26)
centerx_flux =(xlim_flux[1:]+xlim_flux[:-1])/2
centery_flux =(ylim_flux[1:]+ylim_flux[:-1])/2
centerz_flux =(zlim_flux[1:]+zlim_flux[:-1])/2

kT=1
f, ax = plt.subplots(1,2,figsize=(9.5,5), sharey=True)
i=4
plot_pmf(np.asarray(cv1),np.asarray(cv2),np.asarray(C_COMInds[i]),kT,xlim,ylim,ax=ax[0],vmax=12.5,
                title=None, label=None,
                xlabel='Pathway 1 Contacts', ylabel='Pathway 4 Contacts', contours=True, pcolor=False)
plot_pmf(np.asarray(cv1),np.asarray(cv2),np.asarray(C_COMInds[i]),kT,xlim,ylim,ax=ax[1],vmax=12.5,
                title=None, label=None,
                xlabel='Pathway 1 Contacts', ylabel='Pathway 4 Contacts', contours=True, pcolor=False)
kde, kde1, e1, e2 = kdesum2d(cv1,cv2,np.asarray(JBAx[i]),np.asarray(JBAy[i]),xmin=0,xmax=100,ymin=0,ymax=100,nx=22,ny=22,xstd=5,ystd=5,plot=True,axis=ax[0],
                xlabel='Pathway 1 Contacts', ylabel='Pathway 4 Contacts', title='Binding', cbar = True)
kde, kde1, e1, e2 = kdesum2d(cv1,cv2,np.asarray(JABx[i]),np.asarray(JABy[i]),xmin=0,xmax=100,ymin=0,ymax=100,nx=22,ny=22,xstd=5,ystd=5,plot=True,axis=ax[1],
                xlabel='Pathway 1 Contacts', title='Unbinding', cbar=True)
ax[0].set_ylabel('Pathway 4 Contacts', fontsize=16)
ax[0].set_title('Binding', fontsize=16)
ax[1].set_title('Unbinding', fontsize=16)
for axis in ax.ravel():
    axis.tick_params(width=1.5, labelsize=14)
    axis.set_xlabel('Pathway 1 Contacts', fontsize=16)

f.tight_layout()

In [ ]:
f, axes = plt.subplots(5,2,figsize=(7,15))

for i, (lag, ax, q, com) in enumerate(zip(lags, axes.flatten(), C_QIndsUnfold, C_COMInds)):
    if i < len(lags):
        plot_avg(np.asarray(cv1),np.asarray(cv2),np.asarray(q),np.asarray(com),xlim,ylim,ax=ax,vmin=0,vmax=1,
                title="Lag=" + str(lag*10) + " ps",label=r'$q_{\mathrm{unbind}}$',
                xlabel=r'$N_{\mathrm{PW1}}$', ylabel=r'$N_{\mathrm{PW1}}$', cmap='coolwarm',
                contours=True, clines=np.linspace(0,1.0,11),q=True)
f.tight_layout()

f.savefig('Paper_figs/q_varyt.png', dpi=1000)

In [ ]:
%matplotlib notebook
q = np.asarray(C_QIndsFold[4])
q_arr = np.asarray(q).flatten()
fig = plt.figure(figsize=(6,6))
ax = plt.axes(projection='3d')
h= ax.scatter(CV_arr[::40,4], CV_arr[::40,5], q_arr[::40], c=q_arr[::40], alpha=0.15, cmap = 'coolwarm')
ax.tick_params(width=1.5, labelsize=12)
ax.set_xlabel(r'$N_{\mathrm{PW1}}$', fontsize=16)
ax.set_ylabel(r'$N_{\mathrm{PW4}}$', fontsize=16)
ax.set_zlabel(r'$q_{\mathrm{bind}}$', fontsize=16)
ax.view_init(elev=27., azim=-141)


fig.tight_layout()


In [ ]:
#Bin and smooth the reactive current in 3 dimensions

i=4
nx=51
ny=51
nz=51
xlim_flux = np.linspace(0,100,nx)
ylim_flux = np.linspace(0,100,ny)
zlim_flux = np.linspace(0,100,nz)
centerx_flux =(xlim_flux[1:]+xlim_flux[:-1])/2
centery_flux =(ylim_flux[1:]+ylim_flux[:-1])/2
centerz_flux =(zlim_flux[1:]+zlim_flux[:-1])/2
cv1 = np.asarray([col[:,4] for col in cv_trajs])
cv4 = np.asarray([col[:,5] for col in cv_trajs])
q = np.asarray(C_QIndsUnfold[i])*100.0
q_arr = np.asarray(q).flatten()

kde1, kde2, kde3, e1, e2, e3 = kdesum3d(cv1,cv4,q,np.asarray(JABx[i]),np.asarray(JABy[i]),np.asarray(qJABz[i])*100.0,
                                        nx=nx-1,ny=ny-1,nz=nz-1,xstd=8,ystd=8,zstd=8,plot=False)

j = kde3.swapaxes(2,0)[20]

In [ ]:
#Plot slices of this as a function of committor

f, axes = plt.subplots(4,4,figsize=(9.5,9.5), sharex=True, sharey=True)
for i, (x, ax) in enumerate(zip(kde3.swapaxes(2,0)[8:],axes.ravel())):
    pcm = ax.pcolormesh(xlim_flux, ylim_flux, x, shading="flat", vmax=(np.mean(x) + 3*np.std(x)))
    plt.colorbar(pcm, ax=ax, format="%.0e")
    ax.set_title("q=" + "{:.2f}".format((centerz_flux[i + 8])/100))
    j,k = divmod(i,4)
    if k==0: 
        ax.set_ylabel(r'$N_{\mathrm{PW4}}$')
    if j==3:
        ax.set_xlabel(r'$N_{\mathrm{PW1}}$')
    print( (np.sum(x)))
    ax.axvline(x=80, c='C1')
    ax.axvline(x=44, c='C1')
    ax.axhline(y=70, c='C3')
    ax.axhline(y=32, c='C3')
f.tight_layout()

In [ ]:
#Assign different regions to different pathways

assignments = np.zeros(np.shape(kde3.swapaxes(2,0)[0]))
X, Y = np.meshgrid(centerx_flux, centery_flux)
print(X.shape)
print(assignments.shape)
x_1 = 64
y_1 = 70

x_2 = 40
y_2 = 32

x_3 = 20
y_3 = 16
pw_4a = (Y > y_1)
pw_1a = np.logical_and(pw_4a==False, X > x_1)
pw_4b = np.logical_and(pw_1a==False, np.logical_and(Y > y_2, pw_4a==False))
pw_1b = np.logical_and(pw_4b==False, np.logical_and(pw_1a==False, np.logical_and(X > x_2, pw_4a==False)))
pw_3 = np.logical_and(X < x_3, Y < y_3)

pw_2 = np.logical_and(pw_3==False, np.logical_and(pw_1b==False, np.logical_and(pw_1a==False, 
                                                                               np.logical_and(pw_4a==False, pw_4b==False))))
assignments[pw_4a] = 1
assignments[pw_1a] = 2
assignments[pw_4b] = 3
assignments[pw_1b] = 4
assignments[pw_3] = 5
assignments[pw_2] = 6
f, ax =plt.subplots(figsize=(6,5))
pcm = ax.pcolormesh(xlim_flux, ylim_flux, assignments,shading="flat")
plt.colorbar(pcm, ax=ax, format="%.0e")
f.tight_layout()

In [ ]:
#Actually calculate the weights by calculating the appropriate dot products
pw1a_ws = []
pw4a_ws = []
pw1b_ws = []
pw4b_ws = []
pw2_ws = []
pw3_ws = []

for j in kde3.swapaxes(2,0)[8:28]:
    total_flux = np.sum(j)
    pw1a_w = np.sum(pw_1a*j) / total_flux
    pw4a_w = np.sum(pw_4a*j) / total_flux
    pw1b_w = np.sum(pw_1b*j) / total_flux
    pw4b_w = np.sum(pw_4b*j) / total_flux
    pw2_w = np.sum(pw_2*j) / total_flux
    pw3_w = np.sum(pw_3*j) / total_flux
    
    pw1a_ws.append(pw1a_w)
    pw4a_ws.append(pw4a_w)
    pw1b_ws.append(pw1b_w)
    pw4b_ws.append(pw4b_w)
    pw2_ws.append(pw2_w)
    pw3_ws.append(pw3_w)
    
pw1a_ws = np.array(pw1a_ws)
pw4a_ws = np.array(pw4a_ws)
pw1b_ws = np.array(pw1b_ws)
pw4b_ws = np.array(pw4b_ws)
pw2_ws = np.array(pw2_ws)
pw3_ws = np.array(pw3_ws)
print(pw1a_ws + pw1b_ws + pw2_ws + pw3_ws + pw4a_ws + pw4b_ws)
print(pw1a_ws[3], pw1b_ws[3], pw2_ws[3], pw3_ws[3], pw4a_ws[3], pw4b_ws[3])

In [ ]:
#Plot relative weights as a function of the dividing surface q (for one value of lag time). 
#You could also do this for one dividing surface as a function of lag time. 

f, ax = plt.subplots(figsize=(6,6))
ax.plot(centerz_flux[8:28] / 100, pw1a_ws, '--o', label="PW1")
ax.plot(centerz_flux[8:28] /100, pw1b_ws, '--o', label="PW1a")
ax.plot(centerz_flux[8:28]/100, pw2_ws, '--o', label="PW2")
ax.plot(centerz_flux[8:28]/100, pw3_ws, '--o', label="PW3")
ax.plot(centerz_flux[8:28]/100, pw4a_ws, '--o', label="PW4")
ax.plot(centerz_flux[8:28]/100, pw4b_ws, '--o', label="PW4a")
ax.legend(loc='upper right', fontsize=18)
ax.set_xlabel('Q Slice', fontsize=23)
ax.set_ylabel('Relative weight', fontsize=23)
ax.set_title('WT', fontsize=23)
ax.set_ylim((0,1))
f.tight_layout()